In [1]:
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
ds_train = image_dataset_from_directory('input/train/', label_mode='binary', shuffle=True, image_size=[128, 128],batch_size=64)
ds_valid = image_dataset_from_directory('input/valid/', label_mode='binary', shuffle=True, image_size=[128, 128],batch_size=64)

Found 5117 files belonging to 2 classes.
Found 5051 files belonging to 2 classes.


In [3]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label


ds_train = (ds_train
            .map(convert_to_float)
            .cache()
            .prefetch(buffer_size=tf.data.experimental.AUTOTUNE))
ds_valid = (ds_valid
            .map(convert_to_float)
            .cache()
            .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
            )

In [4]:
from keras.applications.vgg16 import VGG16

from tensorflow import keras
from keras import layers

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(128, 128, 3))
conv_base.trainable = False

model = keras.Sequential([
    conv_base,
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

In [5]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

In [ ]:
history=model.fit(
    ds_train,
    epochs=30,
    verbose=1,
)

Epoch 1/30
80/80 [==============================] - 243s 3s/step - loss: 0.9147 - binary_accuracy: 0.7041
Epoch 2/30
80/80 [==============================] - 223s 3s/step - loss: 0.5292 - binary_accuracy: 0.8397
Epoch 3/30
80/80 [==============================] - 223s 3s/step - loss: 0.4746 - binary_accuracy: 0.8736
Epoch 4/30
80/80 [==============================] - 242s 3s/step - loss: 0.4362 - binary_accuracy: 0.8947
Epoch 5/30
80/80 [==============================] - 239s 3s/step - loss: 0.4145 - binary_accuracy: 0.9017
Epoch 6/30
80/80 [==============================] - 238s 3s/step - loss: 0.3996 - binary_accuracy: 0.9027
Epoch 7/30
80/80 [==============================] - 247s 3s/step - loss: 0.3771 - binary_accuracy: 0.9107
Epoch 8/30
80/80 [==============================] - 253s 3s/step - loss: 0.3582 - binary_accuracy: 0.9199
Epoch 9/30
80/80 [==============================] - 232s 3s/step - loss: 0.3363 - binary_accuracy: 0.9269
Epoch 10/30
80/80 [===========================

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot()